# Read CSV to make diNT PWM used for looking up the scores

In [8]:
import csv

FACS_pwm_dict = {}

#opens csv file, reads into a dictionary
with open('facs_pwm.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        FACS_pwm_dict[row['interaction']] = row['coefficient']

# diNT PWM score TIS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import Seq

import operator
import functools

import math

import re

import numpy as np 
import pylab 
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
from Bio.Alphabet import generic_dna, generic_rna


import numpy
import random
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import generic_rna


import itertools


# compartmentalized functions for extracting sequenes

In [3]:
def get_RNA_seq(filename):
    
    RNA_seq_list = []
    
    for seq_record in SeqIO.parse(filename,"fasta"):
        #may fail b/c of transcribe
        try:
            RNA_seq_list.append(seq_record.seq.transcribe())
        except ValueError:
            RNA_seq_list.append(seq_record.seq)
    return RNA_seq_list
    


In [4]:
def get_first_capital_character(seq):
    for i, c in enumerate(seq):
        if c.isupper():
            return i

In [5]:
def get_first_TIS(RNA_seq_list, species):

    if species == "V":

        v_RNA_seq_list = []
        for v_RNA_seq in RNA_seq_list:
            v_TIS_seq = v_RNA_seq[24:35]
            
            if len(v_TIS_seq) == 11:
                v_RNA_seq_list.append(v_TIS_seq)
            

        return v_RNA_seq_list
    
    else:
        TIS_seq_list = []

        for seq in RNA_seq_list:

            start_codon_index = get_first_capital_character(seq)

            TIS_seq = seq[start_codon_index-6:start_codon_index+5].upper()

            if len(TIS_seq) == 11:

                TIS_seq_list.append(TIS_seq)
            

        return TIS_seq_list

# #V for viral
# #H for human
# #G for gemini
# #A for arabidopsis

# def get_first_TIS(RNA_seq_list,species):
    

            
#     elif species == "H":
        
#         h_RNA_seq_list = []
#         for h_RNA_seq in RNA_seq_list:
#             h_RNA_seq_list.append(h_RNA_seq[30:41])
#         return h_RNA_seq_list
    
#     elif species == "G":
        
#         g_RNA_seq_list = []
#         for g_RNA_seq in RNA_seq_list:
#             g_RNA_seq_list.append(g_RNA_seq[29:40])
#         return g_RNA_seq_list
# #     elif species == "A":
# #         for a_RNA_seq in RNA_seq_list:
# #             v_RNA_seq[24:35]


# key based lookup of FACS

In [6]:
def make_keys(sequence):
    interactions = []
    
    #remove AUG
    stripped_TIS = sequence[:6]+sequence[9:]
    range_2 = list(range(len(stripped_TIS)))

    #make the mono interactions
    for a in range(len(stripped_TIS)):
        monont_pos_key = ((stripped_TIS[a],a),(False,False))
        interactions.append(monont_pos_key)

    #make the dint interactions
    for i in range(len(stripped_TIS)-1):

        range_2.pop(0)
        for j in range_2:
            dint_pos_key = ((stripped_TIS[i],i),(stripped_TIS[j],j))
            interactions.append(dint_pos_key)
    return interactions

In [7]:
def get_seqs(file):
    seqs = get_first_TIS(get_RNA_seq(file[0]),file[1])
    return seqs

In [8]:


def predict_TIS_eff(seqs):

    interactions_list = []
    for seq in seqs:
        interactions_list.append(make_keys(seq))
    
    seq_eff = []
    
    for seq in interactions_list:
        
        interactions = []
            
        #get the coefficient per interaction
        for interaction in seq:
            interactions.append(float(FACS_pwm_dict[str(interaction)]))
        
        
        seq_eff.append(79.5*functools.reduce(lambda x, y: x*y, interactions))
    
    return seq_eff

    

    

In [9]:
def analysis_pipeline(file):
    return predict_TIS_eff(get_seqs(file))

In [10]:
ex_TIS_seqs = get_RNA_seq("all_possible_TIS.fasta")
ex_eff = predict_TIS_eff(ex_TIS_seqs)
    

In [11]:

sv_filename   = "SCANNING_viral_all_30up_toSTOP_SEP.fasta"
iv_filename   = "IRES_viral_all_30up_toSTOP_SEP.fasta"
lv_filename   = "LEAKY_viral_all_30up_toSTOP_SEP.fasta"
nv_filename   = "NSP_viral_all_30up_toSTOP_SEP.fasta"
rs_filename   = "RIBOSOMESHUNTING_viral_all_30up_toSTOP_SEP.fasta"
ti_filename   = "TERMINATIONREINITIATION_viral_all_30up_toSTOP_SEP.fasta"
vp_filename   = "VPG_viral_all_30up_toSTOP_SEP.fasta"
uk_filename   = "UNKNOWN_viral_all_30up_toSTOP_SEP.fasta"
hs_filename   = "hs_all_nodup_u36.fasta"
ge_filename   = "YA_diNT_replace_C_gemini_5utr_orf.fasta"
ab_filename   = "ar_utr_cds.fasta"


samples=[ (sv_filename, "V"),
          (iv_filename,"V"),
          (lv_filename,"V"),
          (nv_filename,"V"),
          (rs_filename,"V"),
          (ti_filename,"V"),
          (vp_filename,"V"),
          (uk_filename,"V"),
          (hs_filename,"H"),
          (ge_filename,"G"),
          (ab_filename,"A")
          ]



In [12]:


sv_eff   = analysis_pipeline(samples[0])
iv_eff   = analysis_pipeline(samples[1])
lv_eff   = analysis_pipeline(samples[2])
nv_eff   = analysis_pipeline(samples[3])
rs_eff   = analysis_pipeline(samples[4])
ti_eff   = analysis_pipeline(samples[5])
vp_eff   = analysis_pipeline(samples[6])
uk_eff   = analysis_pipeline(samples[7])
hs_eff   = analysis_pipeline(samples[8])
ge_eff   = analysis_pipeline(samples[9])
ab_eff   = analysis_pipeline(samples[10])




In [13]:
sample_effs = [(sv_eff,"Scanning"),(iv_eff,"IRES"),(lv_eff,"Leaky Scanning"),
               (nv_eff,"NSP3"),(rs_eff,"Ribosome Shunting"),(ti_eff,"Termination-Reinitiation"),(vp_eff,"VPg"),
               (uk_eff,"Unknown"),(hs_eff,"Human"),(ge_eff,"Geminivirus"),(ex_eff,"All possible TIS")]

In [14]:
# above TIS_lists will be the input of below function

# NEGATIVE Controls internal AUG

In [15]:
def extract_after_first_TIS(seqs, species):

    if species == "V":
        index = 35
      
        v_after_first_START = []
    
        for seq in seqs:
            v_after_first_START.append(seq[index:])
    
        return v_after_first_START

    else:
        after_TIS_seqs = []

        for seq in seqs:
            start_codon_index = get_first_capital_character(seq)

            after_TIS_seq = seq[start_codon_index+5:].upper()

            after_TIS_seqs.append(after_TIS_seq)
            
        return after_TIS_seqs

# #cut RNA seqs to after the first TIS
# def extract_after_first_TIS(seqs, species):
    
#     if species == "V":
#         index = 35
#     elif species == "H":
#         index = 40
#     elif species == "G":
#         index = 41
    
#     no_first_seq_list = []
    
#     for seq in seqs:
#         no_first_seq_list.append(seq[index:])
    
#     return no_first_seq_list 


In [16]:
def get_all_downstream_CODON_TIS_indices(sequence,query_codon):
    
    #gets all the indices of the querycodon's first character 
    AUG_positions = [m.start() for m in re.finditer(query_codon,sequence)]
    return AUG_positions
    

In [17]:

# def internal_AUG_analysis_pipeline(file):
    
#     nucleotides = set('AUCG')
#     downstream_TISs = []

#     downstream_seq = extract_after_first_TIS(get_RNA_seq(file[0]), file[1])
    
#     for seq in downstream_seq:
#         downstream_AUGs = get_all_downstream_CODON_TIS_indices(str(seq),"AUG")
    
#         for pos in downstream_AUGs:
#             internal_AUG = seq[pos-6:pos+5]
            
#             re.match('',)
#             if "N" not in internal_AUG:
#                 if "S" not in internal_AUG:
#                     if "K" not in internal_AUG:
#                         if len(internal_AUG) == 11:
#                             downstream_TISs.append(internal_AUG)

#     downstream_diNT_tuples = diNT_tuple_gen(downstream_TISs)
    
#     downstream_PWM_scores = score_seq_with_PWM(downstream_diNT_tuples)
#     efficiencies = convert_to_Pandas_series_adjust_K_val(downstream_PWM_scores)
# #     sns.distplot(efficiencies)
# #     sns.plt.title(file)
# #     plt.show()
#     return efficiencies

    


def internal_AUG_analysis_pipeline(file):
    downstream_TISs = []

    downstream_seq = extract_after_first_TIS(get_RNA_seq(file[0]), file[1])
    
    for seq in downstream_seq:
        downstream_AUGs = get_all_downstream_CODON_TIS_indices(str(seq),"AUG")
    
        for pos in downstream_AUGs:
            internal_AUG = seq[pos-6:pos+5]
            
            if "N" not in internal_AUG:
                if "S" not in internal_AUG:
                    if "K" not in internal_AUG:
                        if len(internal_AUG) == 11:
                            downstream_TISs.append(internal_AUG)
                        

    downstream_TIS_keys = make_keys(downstream_TISs)
    
    efficiencies = analysis_pipeline(downstream_TIS_keys)
    
    return efficiencies

    

In [18]:

sv_IA_eff   = internal_AUG_analysis_pipeline(samples[0])
iv_IA_eff   = internal_AUG_analysis_pipeline(samples[1])
lv_IA_eff   = internal_AUG_analysis_pipeline(samples[2])
nv_IA_eff   = internal_AUG_analysis_pipeline(samples[3])
rs_IA_eff   = internal_AUG_analysis_pipeline(samples[4])
ti_IA_eff   = internal_AUG_analysis_pipeline(samples[5])
vp_IA_eff   = internal_AUG_analysis_pipeline(samples[6])
uk_IA_eff   = internal_AUG_analysis_pipeline(samples[7])
hs_IA_eff   = internal_AUG_analysis_pipeline(samples[8])
ge_IA_eff   = internal_AUG_analysis_pipeline(samples[9])


KeyboardInterrupt: 

In [ ]:
internal_AUG_eff = [(sv_IA_eff,"Scanning - int AUG"),(iv_IA_eff,"IRES- int AUG"),(lv_IA_eff,"Leaky Scanning- int AUG"),
                    (nv_IA_eff,"NSP3- int AUG"),(rs_IA_eff,"Ribosome Shunting- int AUG"),(ti_IA_eff,"Termination-Reinitiation- int AUG"),(vp_IA_eff,"VPg- int AUG"),
                    (uk_IA_eff,"Unknown- int AUG"),(hs_IA_eff,"Human- int AUG"),(ge_IA_eff,"Geminivirus- int AUG")]

# Scramble and PWMscore  negative control

In [ ]:
def scramble_TIS(TIS_list):
    #rerun to get more random seq
    shuffled_TIS_list =[]

    for seq in TIS_list:

        #get the TIS only
        TIS_only = str(seq[0:6] + seq[9:11])

        #shuffles the TIS string
        shuffled_TIS_source_string = ''.join(random.sample(TIS_only,len(TIS_only)))

        shuffled_record = Seq(shuffled_TIS_source_string[0:6] + "AUG" + shuffled_TIS_source_string[6:8], 
                generic_rna)
        shuffled_TIS_list.append(shuffled_record)
    return shuffled_TIS_list
    

In [ ]:
def scramble_pipeline(file):

    seqs = get_first_TIS(get_RNA_seq(file[0]),file[1])
    scram_seqs = scramble_TIS(seqs)
    efficiencies = predict_TIS_eff(scram_seqs)
    
    return efficiencies
    
    

In [ ]:

sv_scram_eff   = scramble_pipeline(samples[0])
iv_scram_eff   = scramble_pipeline(samples[1])
lv_scram_eff   = scramble_pipeline(samples[2])
nv_scram_eff   = scramble_pipeline(samples[3])
rs_scram_eff   = scramble_pipeline(samples[4])
ti_scram_eff   = scramble_pipeline(samples[5])
vp_scram_eff   = scramble_pipeline(samples[6])
uk_scram_eff   = scramble_pipeline(samples[7])
hs_scram_eff   = scramble_pipeline(samples[8])
ge_scram_eff   = scramble_pipeline(samples[9])


In [ ]:
scram_eff = [(sv_scram_eff,"Scanning - int AUG"),(iv_scram_eff,"IRES- int AUG"),(lv_scram_eff,"Leaky Scanning- int AUG"),
             (nv_scram_eff,"NSP3- int AUG"),(rs_scram_eff,"Ribosome Shunting- int AUG"),(ti_scram_eff,"Termination-Reinitiation- int AUG"),(vp_scram_eff,"VPg- int AUG"),
             (uk_scram_eff,"Unknown- int AUG"),(hs_scram_eff,"Human- int AUG"),(ge_scram_eff,"Geminivirus- int AUG")]

In [5]:
import scipy
import matplotlib.pyplot as plt

# sample_effs
# internal_AUG_eff
# scram_eff


# dens_plots = []
# for eff in viral_eff:
#      dens_plots.append(sns.distplot(eff, hist=False))






# all_viral_test_eff = analysis_pipeline(("viral_all_30up_toSTOP.fasta","V"))


sns.distplot(ex_eff, hist = False, label = "All TIS seq")
sns.distplot(hs_eff, hist = False, label = "human")
plt.show()

# plt.figure(2)
sns.distplot(sv_eff, hist = False, label = "scanning")
sns.distplot(hs_eff, hist = False, label = "human")
plt.show()

# plt.figure(3)
sns.distplot(ab_eff, hist = False, label = "arabidopsis")
sns.distplot(ge_eff, hist = False, label = "geminivirus")

# plt.ylabel('Density')
# plt.xlabel('TIS Efficiency')


plt.show()

# ax = plt.subplot(311)



# scipy.stats.ks_2samp(sample1, sample2)

# #https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html
# scipy.stats.chisquare(hs_eff,ex_eff)

NameError: name 'sns' is not defined

# use the new scoring scheme to scan the entire ORF like what you did in last semester.

In [4]:
# have to scan the TIS + 200 bp downstream
